# **Predicting High-Cost Medicare Beneficiaries: A Machine Learning Comparison Using CMS Synthetic Claims Data**

Medicare covers 65M+ beneficiaries and exceeds $100B in annual Part D spending, with a small subset of patients driving a disproportionate share of total costs. Early identification of these high-cost beneficiaries enables targeted care management, supports value-based care contracts, and informs specialty drug program design.

This project uses CMS De-SynPUF Sample 9, a 0.25% subsample containing ~115k beneficiaries per year (2008–2010) and millions of associated claims, including 4.7M Carrier, 66k inpatient, 791k outpatient, and 5.55M PDE records. These files span inpatient, outpatient, professional, and pharmacy utilization to support comprehensive feature development.

Although the dataset is synthetic and based on ICD-9–era patterns, its structure closely mirrors real CMS claims, making it useful for methodological testing. The study evaluates: (1) characteristics of high-cost beneficiaries, (2) which ML model (among logistic regression, tree-based models, and neural network) performs best, (3) key cost-driving features, (4) how well synthetic data reflects real Medicare patterns, and (5) implications for value-based care.

# **Part 1: Data Cleaning**

In this section, we'll be importing the relevant libraries, standardizing values, handling potential missingness, and curating a benficiary-level dataset. The CMS Beneficiary Summary file has 32 variables, CMS Inpatient Claims have 81 variables, Outpatient Claims have 76 variables, Carrier Claims have 142 variables, and Prescription Drug Events have 8 variables.

In [1]:
import numpy as np
import pandas as pd
import os
import pickle
import re

# **1.1 Beneficiaries**

In [28]:
beneficiaries_all = pd.read_pickle("data/beneficiaries_raw.pkl")

In [29]:
beneficiaries_all.shape

(343467, 33)

In [30]:
beneficiaries_all.columns

Index(['DESYNPUF_ID', 'BENE_BIRTH_DT', 'BENE_DEATH_DT', 'BENE_SEX_IDENT_CD',
       'BENE_RACE_CD', 'BENE_ESRD_IND', 'SP_STATE_CODE', 'BENE_COUNTY_CD',
       'BENE_HI_CVRAGE_TOT_MONS', 'BENE_SMI_CVRAGE_TOT_MONS',
       'BENE_HMO_CVRAGE_TOT_MONS', 'PLAN_CVRG_MOS_NUM', 'SP_ALZHDMTA',
       'SP_CHF', 'SP_CHRNKIDN', 'SP_CNCR', 'SP_COPD', 'SP_DEPRESSN',
       'SP_DIABETES', 'SP_ISCHMCHT', 'SP_OSTEOPRS', 'SP_RA_OA', 'SP_STRKETIA',
       'MEDREIMB_IP', 'BENRES_IP', 'PPPYMT_IP', 'MEDREIMB_OP', 'BENRES_OP',
       'PPPYMT_OP', 'MEDREIMB_CAR', 'BENRES_CAR', 'PPPYMT_CAR', 'year'],
      dtype='object')

In [31]:
beneficiaries_all.head()

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,...,MEDREIMB_IP,BENRES_IP,PPPYMT_IP,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR,year
0,000102649ED5601B,19311001,NaN,2,1,0,49,290,12,12,...,0.0,0.0,0.0,360.0,90.0,0.0,860.0,170.0,0.0,2008
1,0002278C944E240A,19360901,NaN,2,1,Y,34,250,12,12,...,0.0,0.0,0.0,2590.0,1530.0,0.0,5000.0,1230.0,0.0,2008
2,000330E625C93700,19330901,NaN,2,1,0,4,420,12,12,...,0.0,0.0,0.0,0.0,0.0,0.0,270.0,150.0,0.0,2008
3,000374D5E110EDA6,19360801,NaN,2,1,0,5,90,12,12,...,0.0,0.0,0.0,60.0,30.0,0.0,1090.0,360.0,0.0,2008
4,0003950E4B4FEC8D,19401101,NaN,2,2,0,19,420,12,12,...,0.0,0.0,0.0,3690.0,420.0,0.0,3590.0,960.0,0.0,2008


In [32]:
patient_counts = len(beneficiaries_all["DESYNPUF_ID"].unique())
beneficiaries_all["DESYNPUF_ID"].value_counts()

DESYNPUF_ID
000102649ED5601B    3
A8F969E2702A45C8    3
A8FFD5B88B46E7EC    3
A8FF09E5AE205085    3
A8FE4D55804CDB15    3
                   ..
F1D939CDE80F890B    1
0D37D9F36731CCE8    1
0D2910F8F2E46ED5    1
B10FF9F39B07D609    1
6DA8C92253DF07C6    1
Name: count, Length: 116287, dtype: int64

In [33]:
beneficiaries_all[beneficiaries_all["DESYNPUF_ID"] == "000102649ED5601B"]

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,...,MEDREIMB_IP,BENRES_IP,PPPYMT_IP,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR,year
0,000102649ED5601B,19311001,NaN,2,1,0,49,290,12,12,...,0.0,0.0,0.0,360.0,90.0,0.0,860.0,170.0,0.0,2008
116287,000102649ED5601B,19311001,NaN,2,1,0,49,290,12,12,...,0.0,0.0,0.0,210.0,70.0,0.0,560.0,170.0,0.0,2009
230782,000102649ED5601B,19311001,NaN,2,1,0,49,290,12,12,...,0.0,0.0,0.0,0.0,0.0,0.0,820.0,170.0,0.0,2010


This shows that the same id pops up multiple times to correspond with the years on file. It would be great for feature engineering to see what values changed for the same patient over the years. It'll also be interesting to observe for those that only had 2 years on file and reached death, what changed and what stayed constant.

In [34]:
# IDs that appear exactly 2 times
counts = beneficiaries_all['DESYNPUF_ID'].value_counts()
ids_2_times = counts[counts == 2].index

# Filter to those IDs
df_2_years = beneficiaries_all[beneficiaries_all['DESYNPUF_ID'].isin(ids_2_times)]

# 3 IDs that did NOT die (BENE_DEATH_DT is NaN)
alive_ids = df_2_years[df_2_years['BENE_DEATH_DT'].isna()]['DESYNPUF_ID'].unique()[:3]

# 3 IDs that DID die (BENE_DEATH_DT is NOT NaN)
dead_ids = df_2_years[df_2_years['BENE_DEATH_DT'].notna()]['DESYNPUF_ID'].unique()[:3]

In [35]:
# Initialize empty list to track variables that changed
all_var_cols = []

# Check out what values might change for the same patient over the year
ids_to_check = [
    '000102649ED5601B',
    'A8F969E2702A45C8',
    'A8FFD5B88B46E7EC'
] + alive_ids.tolist() + dead_ids.tolist()

for bene_id in ids_to_check:
    group = beneficiaries_all[beneficiaries_all['DESYNPUF_ID'] == bene_id]
    # Identify columns that vary (excluding YEAR)
    var_cols = [col for col in group.columns if group[col].nunique() > 1 and col != 'year']
    
    # Add new variable names to master list
    for col in var_cols:
        if col not in all_var_cols:
            all_var_cols.append(col)

    # Display table if there are varying columns
    if var_cols:
        print(f"\nDESYNPUF_ID: {bene_id}")
        display(group[['year'] + var_cols])
    else:
        print(f"\nDESYNPUF_ID: {bene_id} has no varying columns across years.")

print("\nAggregate list of variables that changed across all selected IDs:")
print(all_var_cols)



DESYNPUF_ID: 000102649ED5601B


,year,SP_COPD,SP_DEPRESSN,SP_DIABETES,SP_ISCHMCHT,SP_OSTEOPRS,SP_STRKETIA,MEDREIMB_OP,BENRES_OP,MEDREIMB_CAR
0,2008,1,2,2,2,1,1,360.0,90.0,860.0
116287,2009,2,2,1,1,1,2,210.0,70.0,560.0
230782,2010,2,1,1,1,2,2,0.0,0.0,820.0



DESYNPUF_ID: A8F969E2702A45C8 has no varying columns across years.

DESYNPUF_ID: A8FFD5B88B46E7EC


,year,BENE_SMI_CVRAGE_TOT_MONS,SP_CHRNKIDN,SP_CNCR,SP_COPD,SP_DEPRESSN,SP_DIABETES,SP_ISCHMCHT,SP_OSTEOPRS,SP_RA_OA,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR
76902,2008,12,1,1,1,2,2,2,1,1,1680.0,740.0,500.0,3610.0,1530.0,200.0
192008,2009,12,1,2,2,1,1,1,2,2,1560.0,970.0,0.0,3010.0,1190.0,0.0
305321,2010,8,2,2,2,2,2,1,2,2,100.0,0.0,0.0,1280.0,540.0,0.0



DESYNPUF_ID: 002A577D427BF2CA


...


DESYNPUF_ID: 00528A9327C574A8 has no varying columns across years.

DESYNPUF_ID: 002A577D427BF2CA


,year,SP_CHF,SP_COPD,SP_ISCHMCHT,SP_RA_OA,MEDREIMB_IP,BENRES_IP,MEDREIMB_OP,BENRES_OP,MEDREIMB_CAR,BENRES_CAR
80,2008,1,1,1,2,21160.0,1024.0,930.0,410.0,2220.0,600.0
116366,2009,2,2,2,1,0.0,0.0,710.0,230.0,240.0,50.0



DESYNPUF_ID: 004EF7B44A7C5649


,year,SP_ALZHDMTA,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR
141,2008,2,0.0,0.0,0.0,3220.0,720.0,0.0
116426,2009,1,1400.0,600.0,400.0,3200.0,820.0,100.0



DESYNPUF_ID: 00528A9327C574A8 has no varying columns across years.

Aggregate list of variables that changed across all selected IDs:
['SP_COPD', 'SP_DEPRESSN', 'SP_DIABETES', 'SP_ISCHMCHT', 'SP_OSTEOPRS', 'SP_STRKETIA', 'MEDREIMB_OP', 'BENRES_OP', 'MEDREIMB_CAR', 'BENE_SMI_CVRAGE_TOT_MONS', 'SP_CHRNKIDN', 'SP_CNCR', 'SP_RA_OA', 'PPPYMT_OP', 'BENRES_CAR', 'PPPYMT_CAR', 'SP_CHF', 'MEDREIMB_IP', 'BENRES_IP', 'SP_ALZHDMTA']


In [36]:
# Convert BIRTH_DT and BENE_DEATH_DT to datetime
for col in ['BENE_BIRTH_DT', 'BENE_DEATH_DT']:
    beneficiaries_all[col] = pd.to_datetime(beneficiaries_all[col], format='%Y%m%d', errors='coerce')

# Add binary death column
beneficiaries_all['is_dead'] = beneficiaries_all['BENE_DEATH_DT'].notna().astype(int)

# Compute current age per row (death year if exists, else row year)
beneficiaries_all['age'] = beneficiaries_all.apply(
    lambda row: (row['BENE_DEATH_DT'].year if pd.notna(row['BENE_DEATH_DT']) else row['year']) - row['BENE_BIRTH_DT'].year,
    axis=1
)

# Compute baseline age (2008)
baseline_age = beneficiaries_all[beneficiaries_all['year'] == 2008][['DESYNPUF_ID', 'age']]
baseline_age = baseline_age.set_index('DESYNPUF_ID')['age']

# Merge baseline age back
beneficiaries_all['baseline_age'] = beneficiaries_all['DESYNPUF_ID'].map(baseline_age)

# Create bins based on baseline min/max
min_age = beneficiaries_all['baseline_age'].min()
max_age = beneficiaries_all['baseline_age'].max() + (beneficiaries_all['year'].max() - 2008)  # cover full age range

bin_width = 5
bins = list(range(int(min_age), int(max_age) + bin_width, bin_width))

# pd.cut automatically includes the last value if right=True (default)
labels = [f"{bins[i]}-{bins[i+1]-1}" for i in range(len(bins)-1)]

# Assign age bins
beneficiaries_all['age_bin'] = pd.cut(beneficiaries_all['age'], bins=bins, labels=labels, include_lowest=True)

beneficiaries_all = beneficiaries_all.sort_values(by=['DESYNPUF_ID', 'year']).reset_index(drop=True)
beneficiaries_all.head(10)

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,...,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR,year,is_dead,age,baseline_age,age_bin
0,000102649ED5601B,1931-10-01,NaT,2,1,0,49,290,12,12,...,90.0,0.0,860.0,170.0,0.0,2008,0,77,77,75-79
1,000102649ED5601B,1931-10-01,NaT,2,1,0,49,290,12,12,...,70.0,0.0,560.0,170.0,0.0,2009,0,78,77,75-79
2,000102649ED5601B,1931-10-01,NaT,2,1,0,49,290,12,12,...,0.0,0.0,820.0,170.0,0.0,2010,0,79,77,75-79
3,0002278C944E240A,1936-09-01,NaT,2,1,Y,34,250,12,12,...,1530.0,0.0,5000.0,1230.0,0.0,2008,0,72,72,70-74
4,0002278C944E240A,1936-09-01,NaT,2,1,Y,34,250,12,12,...,1470.0,2000.0,2820.0,730.0,0.0,2009,0,73,72,70-74
5,0002278C944E240A,1936-09-01,NaT,2,1,0,34,250,12,12,...,10.0,0.0,2300.0,840.0,0.0,2010,0,74,72,70-74
6,000330E625C93700,1933-09-01,NaT,2,1,0,4,420,12,12,...,0.0,0.0,270.0,150.0,0.0,2008,0,75,75,70-74
7,000330E625C93700,1933-09-01,NaT,2,1,0,4,420,12,12,...,720.0,0.0,780.0,330.0,50.0,2009,0,76,75,75-79
8,000330E625C93700,1933-09-01,NaT,2,1,0,4,420,12,12,...,0.0,0.0,600.0,120.0,0.0,2010,0,77,75,75-79
9,000374D5E110EDA6,1936-08-01,NaT,2,1,0,5,90,12,12,...,30.0,0.0,1090.0,360.0,0.0,2008,0,72,72,70-74


In [37]:
# Map sex and race to strings
beneficiaries_all['sex'] = beneficiaries_all['BENE_SEX_IDENT_CD'].map({1: 'Male', 2: 'Female'}).astype("category")
beneficiaries_all['race'] = beneficiaries_all['BENE_RACE_CD'].map({1: 'White', 2: 'Black', 3: 'Other', 5: 'Hispanic'}).astype("category")

# Convert ESRD indicator to 0/1
beneficiaries_all['bene_esrd'] = beneficiaries_all['BENE_ESRD_IND'].map({'0': 0, 'Y': 1}).astype("category")

# List of chronic condition SP columns to convert to 0/1
sp_columns = [
    'SP_ALZHDMTA', 'SP_CHF', 'SP_CHRNKIDN', 'SP_CNCR', 'SP_COPD', 
    'SP_DEPRESSN', 'SP_DIABETES', 'SP_ISCHMCHT', 'SP_OSTEOPRS', 
    'SP_RA_OA', 'SP_STRKETIA'
]

# Convert each SP column to 0/1 and category
for col in sp_columns:
    beneficiaries_all[col] = beneficiaries_all[col].map({1: 1, 2: 0}).astype('category')

# drop original code columns
beneficiaries_all.drop(columns=['BENE_SEX_IDENT_CD','BENE_RACE_CD','BENE_ESRD_IND'], inplace=True)
beneficiaries_all


,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,BENE_HMO_CVRAGE_TOT_MONS,PLAN_CVRG_MOS_NUM,SP_ALZHDMTA,...,BENRES_CAR,PPPYMT_CAR,year,is_dead,age,baseline_age,age_bin,sex,race,bene_esrd
0,000102649ED5601B,1931-10-01,NaT,49,290,12,12,0,12,1,...,170.0,0.0,2008,0,77,77,75-79,Female,White,0
1,000102649ED5601B,1931-10-01,NaT,49,290,12,12,0,12,1,...,170.0,0.0,2009,0,78,77,75-79,Female,White,0
2,000102649ED5601B,1931-10-01,NaT,49,290,12,12,0,12,1,...,170.0,0.0,2010,0,79,77,75-79,Female,White,0
3,0002278C944E240A,1936-09-01,NaT,34,250,12,12,0,12,1,...,1230.0,0.0,2008,0,72,72,70-74,Female,White,1
4,0002278C944E240A,1936-09-01,NaT,34,250,12,12,0,12,1,...,730.0,0.0,2009,0,73,72,70-74,Female,White,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343462,FFFE4E347796686D,1933-01-01,NaT,50,20,12,12,0,12,0,...,630.0,250.0,2009,0,76,75,75-79,Male,White,1
343463,FFFE4E347796686D,1933-01-01,NaT,50,20,12,12,0,12,0,...,220.0,0.0,2010,0,77,75,75-79,Male,White,1
343464,FFFFEE90E1B4D0DF,1936-12-01,NaT,42,220,12,12,0,0,0,...,190.0,0.0,2008,0,72,72,70-74,Male,White,0
343465,FFFFEE90E1B4D0DF,1936-12-01,NaT,42,220,12,12,0,12,0,...,450.0,0.0,2009,0,73,72,70-74,Male,White,0


In [38]:
reimb_cols = [
    'MEDREIMB_IP', 'BENRES_IP', 'PPPYMT_IP',
    'MEDREIMB_OP', 'BENRES_OP', 'PPPYMT_OP',
    'MEDREIMB_CAR', 'BENRES_CAR', 'PPPYMT_CAR'
]

for col in reimb_cols:
    beneficiaries_all[col] = pd.to_numeric(beneficiaries_all[col], errors='coerce')
    print(col, beneficiaries_all[col].min(), beneficiaries_all[col].max())

MEDREIMB_IP -1000.0 170710.0
BENRES_IP 0.0 38272.0
PPPYMT_IP 0.0 78000.0
MEDREIMB_OP -100.0 53530.0
BENRES_OP 0.0 14050.0
PPPYMT_OP 0.0 16000.0
MEDREIMB_CAR 0.0 19590.0
BENRES_CAR 0.0 5290.0
PPPYMT_CAR 0.0 2500.0


In [39]:
medreimb_cols = ['MEDREIMB_IP', 'MEDREIMB_OP']
for col in medreimb_cols:
    neg_count = (beneficiaries_all[col] < 0).sum()
    total = len(beneficiaries_all)
    print(f"{col}: {neg_count} negative values ({neg_count/total:.2%})")
beneficiaries_all[medreimb_cols] = beneficiaries_all[medreimb_cols].clip(lower=0)

MEDREIMB_IP: 24 negative values (0.01%)
MEDREIMB_OP: 174 negative values (0.05%)


In [40]:
beneficiaries_all.drop(columns=["BENE_BIRTH_DT", "BENE_DEATH_DT", "BENE_COUNTY_CD"], inplace=True)

In [41]:
beneficiaries_all.dtypes

DESYNPUF_ID                   object
SP_STATE_CODE                  int64
BENE_HI_CVRAGE_TOT_MONS        int64
BENE_SMI_CVRAGE_TOT_MONS       int64
BENE_HMO_CVRAGE_TOT_MONS       int64
PLAN_CVRG_MOS_NUM              int64
SP_ALZHDMTA                 category
SP_CHF                      category
SP_CHRNKIDN                 category
SP_CNCR                     category
SP_COPD                     category
SP_DEPRESSN                 category
SP_DIABETES                 category
SP_ISCHMCHT                 category
SP_OSTEOPRS                 category
SP_RA_OA                    category
SP_STRKETIA                 category
MEDREIMB_IP                  float64
BENRES_IP                    float64
PPPYMT_IP                    float64
MEDREIMB_OP                  float64
BENRES_OP                    float64
PPPYMT_OP                    float64
MEDREIMB_CAR                 float64
BENRES_CAR                   float64
PPPYMT_CAR                   float64
year                           int64
i

Negative values indicate adjustments due to overpayment in previous periods. There are 24 negative values in MEDREIMB_IP (0.01%) and 174 negative values in MEDREIMB_OP (0.05%), which is negligible. Set to 0 to equate no payment.

In [42]:
# Create folder if it doesn't exist
folder = "cleaned_data"
os.makedirs(folder, exist_ok=True)

# Save beneficiaries
with open(os.path.join(folder, 'beneficiaries_cleaned.pkl'), 'wb') as f:
    pickle.dump(beneficiaries_all, f)

# **1.2 Inpatient**

In [43]:
inpatient = pd.read_pickle("data/ip_raw.pkl")

In [44]:
inpatient.head()

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,AT_PHYSN_NPI,OP_PHYSN_NPI,...,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45,year
0,0002278C944E240A,992581161620535,1,20090314.0,20090418.0,3400QT,23000.0,0.0,6.987972e+08,8.972194e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
1,0002278C944E240A,992111161633637,1,20090620.0,20090625.0,3400ZQ,46000.0,0.0,4.561865e+09,6.779930e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
2,0002278C944E240A,992231161630505,1,20090620.0,20090725.0,3400BT,27000.0,0.0,6.343428e+09,9.989246e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
3,0002278C944E240A,992261161643487,1,20100501.0,20100503.0,3400VS,3000.0,0.0,8.527411e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
4,000374D5E110EDA6,992261161657815,1,20091230.0,20091230.0,0500VS,9000.0,0.0,3.909066e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0


In [45]:
inpatient_patient_counts = len(inpatient["DESYNPUF_ID"].unique())
inpatient["DESYNPUF_ID"].value_counts()

DESYNPUF_ID
6F93165140F5B702    12
71BAA98B2228DD7E    12
B1142B7B8A446594    12
F3437F8E93752722    12
C07DD040A3DADE6E    11
                    ..
354A8E3581E107CE     1
8F174BED5D7C8FF0     1
354CC590AB7F0AE7     1
35559F1495198416     1
7EBCF0D3D1F972B3     1
Name: count, Length: 37829, dtype: int64

In [46]:
inpatient[inpatient["DESYNPUF_ID"] == "6F93165140F5B702"]

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,AT_PHYSN_NPI,OP_PHYSN_NPI,...,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45,year
29257,6F93165140F5B702,992171161599274,1,20080530.0,20080605.0,2100ZH,2000.0,0.0,1.019754e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
29258,6F93165140F5B702,992511161662864,1,20080618.0,20080626.0,2100ZH,15000.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
29259,6F93165140F5B702,992131161601984,1,20080919.0,20080927.0,2100ZH,5000.0,0.0,7.168180e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
29260,6F93165140F5B702,992141161643878,1,20080929.0,20081001.0,2100WC,11000.0,0.0,3.285453e+09,3.285453e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
29261,6F93165140F5B702,992061161600597,1,20081001.0,20081020.0,2100ZH,11000.0,0.0,7.168180e+09,7.168180e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
29262,6F93165140F5B702,992371161620285,1,20081029.0,20081105.0,2100ZH,3000.0,0.0,7.168180e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
29263,6F93165140F5B702,992491161632355,1,20090104.0,20090110.0,2100WC,20000.0,0.0,6.990841e+09,4.803637e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
29264,6F93165140F5B702,992151161629244,1,20090116.0,20090119.0,0900WC,4000.0,0.0,5.327200e+09,1.643519e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
29265,6F93165140F5B702,992951161652584,1,20090417.0,20090422.0,2100RT,8000.0,0.0,1.136681e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
29266,6F93165140F5B702,992471161653744,1,20090630.0,20090705.0,2100ZH,6000.0,0.0,1.019754e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0


In [47]:
# Dropping columns un-essential for answering the question
inpatient.drop(columns=["AT_PHYSN_NPI", "OP_PHYSN_NPI", "OT_PHYSN_NPI"], inplace=True)
inpatient.head()

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,CLM_ADMSN_DT,ADMTNG_ICD9_DGNS_CD,...,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45,year
0,0002278C944E240A,992581161620535,1,20090314.0,20090418.0,3400QT,23000.0,0.0,20090314,0389,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
1,0002278C944E240A,992111161633637,1,20090620.0,20090625.0,3400ZQ,46000.0,0.0,20090620,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
2,0002278C944E240A,992231161630505,1,20090620.0,20090725.0,3400BT,27000.0,0.0,20090620,4400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
3,0002278C944E240A,992261161643487,1,20100501.0,20100503.0,3400VS,3000.0,0.0,20100501,5589,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
4,000374D5E110EDA6,992261161657815,1,20091230.0,20091230.0,0500VS,9000.0,0.0,20091230,78650,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0


In [48]:
inpatient.columns

Index(['DESYNPUF_ID', 'CLM_ID', 'SEGMENT', 'CLM_FROM_DT', 'CLM_THRU_DT',
       'PRVDR_NUM', 'CLM_PMT_AMT', 'NCH_PRMRY_PYR_CLM_PD_AMT', 'CLM_ADMSN_DT',
       'ADMTNG_ICD9_DGNS_CD', 'CLM_PASS_THRU_PER_DIEM_AMT',
       'NCH_BENE_IP_DDCTBL_AMT', 'NCH_BENE_PTA_COINSRNC_LBLTY_AM',
       'NCH_BENE_BLOOD_DDCTBL_LBLTY_AM', 'CLM_UTLZTN_DAY_CNT',
       'NCH_BENE_DSCHRG_DT', 'CLM_DRG_CD', 'ICD9_DGNS_CD_1', 'ICD9_DGNS_CD_2',
       'ICD9_DGNS_CD_3', 'ICD9_DGNS_CD_4', 'ICD9_DGNS_CD_5', 'ICD9_DGNS_CD_6',
       'ICD9_DGNS_CD_7', 'ICD9_DGNS_CD_8', 'ICD9_DGNS_CD_9', 'ICD9_DGNS_CD_10',
       'ICD9_PRCDR_CD_1', 'ICD9_PRCDR_CD_2', 'ICD9_PRCDR_CD_3',
       'ICD9_PRCDR_CD_4', 'ICD9_PRCDR_CD_5', 'ICD9_PRCDR_CD_6', 'HCPCS_CD_1',
       'HCPCS_CD_2', 'HCPCS_CD_3', 'HCPCS_CD_4', 'HCPCS_CD_5', 'HCPCS_CD_6',
       'HCPCS_CD_7', 'HCPCS_CD_8', 'HCPCS_CD_9', 'HCPCS_CD_10', 'HCPCS_CD_11',
       'HCPCS_CD_12', 'HCPCS_CD_13', 'HCPCS_CD_14', 'HCPCS_CD_15',
       'HCPCS_CD_16', 'HCPCS_CD_17', 'HCPCS_CD_18', 'HC

In [49]:
inpatient.head(5)

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,CLM_ADMSN_DT,ADMTNG_ICD9_DGNS_CD,...,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45,year
0,0002278C944E240A,992581161620535,1,20090314.0,20090418.0,3400QT,23000.0,0.0,20090314,0389,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
1,0002278C944E240A,992111161633637,1,20090620.0,20090625.0,3400ZQ,46000.0,0.0,20090620,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
2,0002278C944E240A,992231161630505,1,20090620.0,20090725.0,3400BT,27000.0,0.0,20090620,4400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
3,0002278C944E240A,992261161643487,1,20100501.0,20100503.0,3400VS,3000.0,0.0,20100501,5589,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
4,000374D5E110EDA6,992261161657815,1,20091230.0,20091230.0,0500VS,9000.0,0.0,20091230,78650,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0


In [50]:
# Convert columns to datetime
for col in ['CLM_FROM_DT', 'CLM_THRU_DT', 'CLM_ADMSN_DT', 'NCH_BENE_DSCHRG_DT']:
    inpatient[col] = pd.to_datetime(inpatient[col], format='%Y%m%d', errors='coerce')

# Compute initial claim duration in hours
inpatient["claim_duration (days)"] = (inpatient["CLM_THRU_DT"] - inpatient["CLM_FROM_DT"]).dt.total_seconds() / (3600 * 24)

# Fill missing claim duration using admission and discharge dates
inpatient['claim_duration (days)'] = (
    (inpatient['CLM_THRU_DT'].fillna(inpatient['NCH_BENE_DSCHRG_DT']) - 
     inpatient['CLM_FROM_DT'].fillna(inpatient['CLM_ADMSN_DT'])
    ).dt.total_seconds() / (3600 * 24)
)
print(inpatient["claim_duration (days)"].value_counts(dropna=False))


claim_duration (days)
3.0     11158
2.0      9995
4.0      8430
1.0      7618
5.0      6063
6.0      4610
7.0      3568
8.0      2693
9.0      1912
10.0     1559
11.0     1181
12.0     1009
0.0       869
13.0      811
14.0      765
35.0      587
15.0      560
16.0      470
17.0      408
18.0      335
19.0      300
20.0      285
21.0      258
22.0      191
23.0      164
24.0      148
25.0      139
26.0      123
28.0      114
27.0      100
29.0       80
31.0       66
30.0       58
32.0       50
34.0       49
33.0       37
Name: count, dtype: int64


In [51]:
# Follow code book instruction where the first two digits are the state code while the 3rd (occasionally 4th) identifies the type of facility
def parse_prvdr_num(prvdr_num):
    prvdr_str = str(prvdr_num)
    state = prvdr_str[:2]
    
    # Check if 3rd character is alpha
    if len(prvdr_str) >= 3 and prvdr_str[2].isalpha():
        category = prvdr_str[2]  # the letter indicates special unit
        block = prvdr_str[:3]    # could use first 3 chars as key
    else:
        category = None
        block = prvdr_str[:3]    # first 3 digits for normal mapping

    return pd.Series({'state_code': state, 'provider_block': block, 'unit_letter': category})


In [52]:
inpatient[['state_code', 'provider_block', 'unit_letter']] = inpatient['PRVDR_NUM'].apply(parse_prvdr_num)

In [53]:
# Map special letters to descriptive unit names
unit_map = {
    'M': "Psychiatric Unit (Critical Access Hospital)",
    'R': "Rehabilitation Unit (Critical Access Hospital)",
    'S': "Psychiatric Unit (excluded from PPS)",
    'T': "Rehabilitation Unit (excluded from PPS)",
    'U': "Swing–Bed Hospital (Short Term)",
    'V': "Alcohol/drug Unit (pre-10/87)",
    'W': "Swing–Bed Hospital (Long Term Care)",
    'Y': "Swing–Bed Hospital (Rehabilitation)",
    'Z': "Swing–Bed Hospital (Critical Access)"
}

def categorize_provider(row):
    num_block = row['provider_block']  
    unit_letter = row['unit_letter']   
    
    # If unit_letter exists and is in our map, use that category
    if pd.notna(unit_letter) and unit_letter in unit_map:
        return unit_map[unit_letter]
    
    # Otherwise, fallback to numeric block mapping
    try:
        num_block = int(num_block)
    except:
        return "Unknown / invalid"

    if 1 <= num_block <= 879:
        category = "Short-term hospital / ESRD clinic"
    elif 880 <= num_block <= 899:
        category = "ORD demonstration hospital"
    elif 900 <= num_block <= 999:
        category = "Multiple hospital component"
    elif 1000 <= num_block <= 1199:
        category = "Reserved / future use"
    elif 1200 <= num_block <= 1224:
        category = "Alcohol/drug hospital"
    elif 1225 <= num_block <= 1299:
        category = "Medical assistance facility"
    elif 1300 <= num_block <= 1399:
        category = "Rural primary care / Critical Access Hospital"
    else:
        category = "Other / Unknown"
    
    return category

# Apply mapping
inpatient['provider_category'] = inpatient.apply(categorize_provider, axis=1)

In [54]:
inpatient["provider_category"].value_counts()

provider_category
Short-term hospital / ESRD clinic                 62324
Rehabilitation Unit (excluded from PPS)            2260
Psychiatric Unit (excluded from PPS)               1812
Unknown / invalid                                   211
Swing–Bed Hospital (Short Term)                      51
Swing–Bed Hospital (Long Term Care)                  28
Rehabilitation Unit (Critical Access Hospital)       25
Alcohol/drug Unit (pre-10/87)                        24
Psychiatric Unit (Critical Access Hospital)          16
Swing–Bed Hospital (Critical Access)                 12
Name: count, dtype: int64

In [55]:
# Check percentage of negative values
total = inpatient.shape[0]
neg_count = (inpatient['CLM_PMT_AMT'] < 0).sum()
neg_pct = neg_count / total * 100
print(f"CLM_PMT_AMT: {neg_count} negative values ({neg_pct:.4f}%)")

# Replace negatives with 0
inpatient['CLM_PMT_AMT'] = inpatient['CLM_PMT_AMT'].clip(lower=0)

CLM_PMT_AMT: 51 negative values (0.0764%)


Negative CLM_PMT_AMT values happen mainly due to Medicare payment adjustments. Sometimes the beneficiary’s deductible or coinsurance exceeds the amount Medicare actually pays, producing a negative. Other times, it reflects reversals or corrections of previous overpayments. Special payment systems, like per-diem rates for rehab or psychiatric units, can also create negatives if the calculated payment is less than the billed amount. Demo programs or bundled payment arrangements may record what Medicare would pay rather than the actual provider payment, leading to negatives. In short, negatives represent adjustments or discrepancies, not actual payments from providers to Medicare.

In [56]:
def summarize_column(df, col, top_n=10):
    """
    Summarizes a column in a dataframe.

    Args:
        df (pd.DataFrame): Dataframe containing the column.
        col (str): Column name to summarize.
        top_n (int): Number of top value counts to display for categorical columns.

    Returns:
        summary (dict): Dictionary with missing count, missing %, min, max, and value_counts.
    """
    missing_count = df[col].isna().sum()
    missing_pct = missing_count / len(df) * 100
    
    summary = {
        'missing_count': missing_count,
        'missing_pct': missing_pct
    }
    
    if pd.api.types.is_numeric_dtype(df[col]):
        summary['min'] = df[col].min()
        summary['max'] = df[col].max()
        if df[col].nunique() <= top_n:
            summary['value_counts'] = df[col].value_counts()
    else:
        summary['value_counts'] = df[col].value_counts().head(top_n)
    
    return summary


In [57]:
cols_to_check = [
    'ADMTNG_ICD9_DGNS_CD',
    'CLM_PASS_THRU_PER_DIEM_AMT',
    'NCH_BENE_IP_DDCTBL_AMT',
    'NCH_BENE_PTA_COINSRNC_LBLTY_AM',
    'NCH_BENE_BLOOD_DDCTBL_LBLTY_AM',
    'CLM_UTLZTN_DAY_CNT',
    'ICD9_DGNS_CD_1',
    'HCPCS_CD_1'
]

summaries = {}
for col in cols_to_check:
    summaries[col] = summarize_column(inpatient, col)

for col, summary in summaries.items():
    print(f"--- {col} ---")
    for k, v in summary.items():
        print(f"{k}: {v}")
    print("\n")


--- ADMTNG_ICD9_DGNS_CD ---
missing_count: 598
missing_pct: 0.8957057052559052
value_counts: ADMTNG_ICD9_DGNS_CD
78650    2781
78605    2716
486      2535
4280     1859
7802     1724
V5789    1491
78097    1472
78079    1230
78900    1210
49121    1101
Name: count, dtype: int64


--- CLM_PASS_THRU_PER_DIEM_AMT ---
missing_count: 0
missing_pct: 0.0
min: 0.0
max: 500.0


--- NCH_BENE_IP_DDCTBL_AMT ---
missing_count: 2159
missing_pct: 3.2338271198118718
min: 1024.0
max: 1100.0
value_counts: NCH_BENE_IP_DDCTBL_AMT
1024.0    26075
1068.0    24802
1100.0    13727
Name: count, dtype: int64


--- NCH_BENE_PTA_COINSRNC_LBLTY_AM ---
missing_count: 0
missing_pct: 0.0
min: 0.0
max: 34000.0


--- NCH_BENE_BLOOD_DDCTBL_LBLTY_AM ---
missing_count: 0
missing_pct: 0.0
min: 0.0
max: 2000.0


--- CLM_UTLZTN_DAY_CNT ---
missing_count: 61
missing_pct: 0.0913679732786124
min: 0.0
max: 150.0


--- ICD9_DGNS_CD_1 ---
missing_count: 94
missing_pct: 0.1407965489867142
value_counts: ICD9_DGNS_CD_1
486      2525


In [58]:
top_n = 10  
top_codes = inpatient["ADMTNG_ICD9_DGNS_CD"].value_counts().head(top_n).index
def map_icd9_to_bucket(code):
    if pd.isna(code):
        return "missing"
    
    if code in ["78605", "486", "49121"]:
        return "respiratory"
    
    if code in ["78650", "4280"]:
        return "cardiac_chest_pain"
    
    if code in ["7802", "78097"]:
        return "neurologic"
    
    if code in ["V5789"]:
        return "renal_dialysis"
    
    if code in ["78900"]:
        return "GI"
    
    if code in ["78079"]:
        return "general_symptoms"

    return "other"

inpatient["ADMTNG_BUCKET"] = inpatient["ADMTNG_ICD9_DGNS_CD"].apply(map_icd9_to_bucket)

In [59]:
def map_primary_icd9_bucket(code):
    if pd.isna(code):
        return "missing"
    
    if code in ["486"]:
        return "respiratory_infection"
    
    if code in ["V5789"]:
        return "renal_dialysis"
    
    if code in ["41401"]:
        return "cardiac_coronary_disease"
    
    if code in ["0389"]:
        return "infectious_sepsis"
    
    if code in ["49121"]:
        return "respiratory_COPD"
    
    if code in ["4280"]:
        return "cardiac_heart_failure"
    
    if code in ["5990"]:
        return "GU_infection"
    
    if code in ["42731"]:
        return "cardiac_arrhythmia"
    
    if code in ["41071"]:
        return "cardiac_MI"
    
    if code in ["5849"]:
        return "renal_acute_kidney_injury"
    
    return "other"

# Apply
inpatient["PRIMARY_ICD9_BUCKET"] = inpatient["ICD9_DGNS_CD_1"].apply(map_primary_icd9_bucket)


In [60]:
inpatient.head(20)

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,CLM_ADMSN_DT,ADMTNG_ICD9_DGNS_CD,...,HCPCS_CD_44,HCPCS_CD_45,year,claim_duration (days),state_code,provider_block,unit_letter,provider_category,ADMTNG_BUCKET,PRIMARY_ICD9_BUCKET
0,0002278C944E240A,992581161620535,1,2009-03-14,2009-04-18,3400QT,23000.0,0.0,2009-03-14,0389,...,NaN,NaN,1970.0,35.0,34,340,None,Short-term hospital / ESRD clinic,other,other
1,0002278C944E240A,992111161633637,1,2009-06-20,2009-06-25,3400ZQ,46000.0,0.0,2009-06-20,NaN,...,NaN,NaN,1970.0,5.0,34,340,None,Short-term hospital / ESRD clinic,missing,other
2,0002278C944E240A,992231161630505,1,2009-06-20,2009-07-25,3400BT,27000.0,0.0,2009-06-20,4400,...,NaN,NaN,1970.0,35.0,34,340,None,Short-term hospital / ESRD clinic,other,other
3,0002278C944E240A,992261161643487,1,2010-05-01,2010-05-03,3400VS,3000.0,0.0,2010-05-01,5589,...,NaN,NaN,1970.0,2.0,34,340,None,Short-term hospital / ESRD clinic,other,other
4,000374D5E110EDA6,992261161657815,1,2009-12-30,2009-12-30,0500VS,9000.0,0.0,2009-12-30,78650,...,NaN,NaN,1970.0,0.0,05,050,None,Short-term hospital / ESRD clinic,cardiac_chest_pain,other
5,0003950E4B4FEC8D,992391161625071,1,2009-09-25,2009-10-04,1900DN,25000.0,0.0,2009-09-25,V5789,...,NaN,NaN,1970.0,9.0,19,190,None,Short-term hospital / ESRD clinic,renal_dialysis,renal_dialysis
6,0003950E4B4FEC8D,992541161645706,1,2010-04-30,2010-05-01,19016K,3000.0,0.0,2010-04-30,78650,...,NaN,NaN,1970.0,1.0,19,190,None,Short-term hospital / ESRD clinic,cardiac_chest_pain,other
7,0005FE4AE779FCD7,992731161618503,1,2008-07-19,2008-07-22,1501HB,10000.0,0.0,2008-07-19,71535,...,NaN,NaN,1970.0,3.0,15,150,None,Short-term hospital / ESRD clinic,other,other
8,0005FE4AE779FCD7,992281161634506,1,2009-04-28,2009-05-01,1501HB,15000.0,0.0,2009-04-28,71536,...,NaN,NaN,1970.0,3.0,15,150,None,Short-term hospital / ESRD clinic,other,other
9,00072304F3E15CAD,992931161654683,1,2008-07-27,2008-07-29,3600NG,5000.0,0.0,2008-07-27,7802,...,NaN,NaN,1970.0,2.0,36,360,None,Short-term hospital / ESRD clinic,neurologic,renal_acute_kidney_injury


In [61]:
# Deductible is almost always 0 when there's a missing value, so we don't impute but set to 0.
inpatient["NCH_BENE_IP_DDCTBL_AMT"] = inpatient["NCH_BENE_IP_DDCTBL_AMT"].fillna(0)

We bucket utilization days into clinically meaningful hospitalization lengths:
1. Very Short (0–2 days) — observation-like or uncomplicated short stays.
2. Short (3–7 days) — typical acute inpatient hospital stays.
3. Medium (8–14 days) — more complex acute admissions or early rehab.
4. Long (15–30 days) — rehab, psych, or complicated hospital courses.
5. Extended (31–60 days) — prolonged rehab or long psychiatric stays.
6. Very Long (61–120 days) — long-term acute hospital (LTCH) or chronic critical illness.
7. Extreme (>120 days) — rare, likely synthetic outliers; kept separate to avoid skewing.

In [62]:
def bucket_utilization_days(x):
    if pd.isna(x):
        return "Missing"
    if x <= 2:
        return "Very Short (0–2)"
    elif x <= 7:
        return "Short (3–7)"
    elif x <= 14:
        return "Medium (8–14)"
    elif x <= 30:
        return "Long (15–30)"
    elif x <= 60:
        return "Extended (31–60)"
    elif x <= 120:
        return "Very Long (61–120)"
    else:
        return "Extreme (>120)"

inpatient['derived_days'] = (
    (inpatient['NCH_BENE_DSCHRG_DT'] - inpatient['CLM_ADMSN_DT'])
     .dt.days
)
inpatient['CLM_UTLZTN_DAY_CNT'] = inpatient['CLM_UTLZTN_DAY_CNT'].fillna(inpatient['derived_days'])
inpatient["UTIL_DAY_BUCKET"] = inpatient["CLM_UTLZTN_DAY_CNT"].apply(bucket_utilization_days)
inpatient["UTIL_DAY_BUCKET"].value_counts()

UTIL_DAY_BUCKET
Short (3–7)           32948
Very Short (0–2)      19891
Medium (8–14)          9656
Long (15–30)           3568
Extended (31–60)        622
Very Long (61–120)       75
Extreme (>120)            3
Name: count, dtype: int64

In [63]:
inpatient['year'] = inpatient['CLM_FROM_DT'].dt.year
# The entries with 2007 usually extend treatments into 2008 so lumping them together
inpatient['year'] = inpatient['year'].replace(2007, 2008)

In [64]:
inpatient.drop(columns=['CLM_DRG_CD', 'ICD9_DGNS_CD_1', 'ICD9_DGNS_CD_2',
       'ICD9_DGNS_CD_3', 'ICD9_DGNS_CD_4', 'ICD9_DGNS_CD_5', 'ICD9_DGNS_CD_6',
       'ICD9_DGNS_CD_7', 'ICD9_DGNS_CD_8', 'ICD9_DGNS_CD_9', 'ICD9_DGNS_CD_10',
       'ICD9_PRCDR_CD_1', 'ICD9_PRCDR_CD_2', 'ICD9_PRCDR_CD_3',
       'ICD9_PRCDR_CD_4', 'ICD9_PRCDR_CD_5', 'ICD9_PRCDR_CD_6', 'HCPCS_CD_1',
       'HCPCS_CD_2', 'HCPCS_CD_3', 'HCPCS_CD_4', 'HCPCS_CD_5', 'HCPCS_CD_6',
       'HCPCS_CD_7', 'HCPCS_CD_8', 'HCPCS_CD_9', 'HCPCS_CD_10', 'HCPCS_CD_11',
       'HCPCS_CD_12', 'HCPCS_CD_13', 'HCPCS_CD_14', 'HCPCS_CD_15',
       'HCPCS_CD_16', 'HCPCS_CD_17', 'HCPCS_CD_18', 'HCPCS_CD_19',
       'HCPCS_CD_20', 'HCPCS_CD_21', 'HCPCS_CD_22', 'HCPCS_CD_23',
       'HCPCS_CD_24', 'HCPCS_CD_25', 'HCPCS_CD_26', 'HCPCS_CD_27',
       'HCPCS_CD_28', 'HCPCS_CD_29', 'HCPCS_CD_30', 'HCPCS_CD_31',
       'HCPCS_CD_32', 'HCPCS_CD_33', 'HCPCS_CD_34', 'HCPCS_CD_35',
       'HCPCS_CD_36', 'HCPCS_CD_37', 'HCPCS_CD_38', 'HCPCS_CD_39',
       'HCPCS_CD_40', 'HCPCS_CD_41', 'HCPCS_CD_42', 'HCPCS_CD_43',
       'HCPCS_CD_44', 'HCPCS_CD_45', "PRVDR_NUM", "ADMTNG_ICD9_DGNS_CD", "provider_block", "CLM_ADMSN_DT", "NCH_BENE_DSCHRG_DT", "unit_letter", "derived_days"
                       ], inplace=True)


In [65]:
for col in ['provider_category', 'ADMTNG_BUCKET', 'UTIL_DAY_BUCKET', 'PRIMARY_ICD9_BUCKET']:
    inpatient[col] = inpatient[col].astype('category')

In [66]:
inpatient.dtypes

DESYNPUF_ID                               object
CLM_ID                                     int64
SEGMENT                                    int64
CLM_FROM_DT                       datetime64[ns]
CLM_THRU_DT                       datetime64[ns]
CLM_PMT_AMT                              float64
NCH_PRMRY_PYR_CLM_PD_AMT                 float64
CLM_PASS_THRU_PER_DIEM_AMT               float64
NCH_BENE_IP_DDCTBL_AMT                   float64
NCH_BENE_PTA_COINSRNC_LBLTY_AM           float64
NCH_BENE_BLOOD_DDCTBL_LBLTY_AM           float64
CLM_UTLZTN_DAY_CNT                       float64
year                                     float64
claim_duration (days)                    float64
state_code                                object
provider_category                       category
ADMTNG_BUCKET                           category
PRIMARY_ICD9_BUCKET                     category
UTIL_DAY_BUCKET                         category
dtype: object

In [67]:
# Save inpatient
with open(os.path.join(folder, 'inpatient_cleaned.pkl'), 'wb') as f:
    pickle.dump(inpatient, f)

# **1.3 Outpatient**

In [68]:
outpatient = pd.read_pickle("data/op_raw.pkl")

In [69]:
outpatient.head()

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,AT_PHYSN_NPI,OP_PHYSN_NPI,...,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45,year
0,000102649ED5601B,338982266168455,1,20080615.0,20080615.0,10016R,10.0,0.0,5.602731e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
1,000102649ED5601B,338322265737261,1,20080619.0,20080619.0,4901RP,50.0,0.0,7.697919e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
2,000102649ED5601B,338522265531487,1,20080818.0,20080818.0,4901RP,300.0,0.0,9.040810e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
3,000102649ED5601B,338102266226170,1,20090404.0,20090404.0,0900SU,90.0,0.0,8.439030e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0
4,000102649ED5601B,338482265489246,1,20090701.0,20090701.0,4900NA,50.0,0.0,9.228519e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0


In [70]:
outpatient.columns

Index(['DESYNPUF_ID', 'CLM_ID', 'SEGMENT', 'CLM_FROM_DT', 'CLM_THRU_DT',
       'PRVDR_NUM', 'CLM_PMT_AMT', 'NCH_PRMRY_PYR_CLM_PD_AMT', 'AT_PHYSN_NPI',
       'OP_PHYSN_NPI', 'OT_PHYSN_NPI', 'NCH_BENE_BLOOD_DDCTBL_LBLTY_AM',
       'ICD9_DGNS_CD_1', 'ICD9_DGNS_CD_2', 'ICD9_DGNS_CD_3', 'ICD9_DGNS_CD_4',
       'ICD9_DGNS_CD_5', 'ICD9_DGNS_CD_6', 'ICD9_DGNS_CD_7', 'ICD9_DGNS_CD_8',
       'ICD9_DGNS_CD_9', 'ICD9_DGNS_CD_10', 'ICD9_PRCDR_CD_1',
       'ICD9_PRCDR_CD_2', 'ICD9_PRCDR_CD_3', 'ICD9_PRCDR_CD_4',
       'ICD9_PRCDR_CD_5', 'ICD9_PRCDR_CD_6', 'NCH_BENE_PTB_DDCTBL_AMT',
       'NCH_BENE_PTB_COINSRNC_AMT', 'ADMTNG_ICD9_DGNS_CD', 'HCPCS_CD_1',
       'HCPCS_CD_2', 'HCPCS_CD_3', 'HCPCS_CD_4', 'HCPCS_CD_5', 'HCPCS_CD_6',
       'HCPCS_CD_7', 'HCPCS_CD_8', 'HCPCS_CD_9', 'HCPCS_CD_10', 'HCPCS_CD_11',
       'HCPCS_CD_12', 'HCPCS_CD_13', 'HCPCS_CD_14', 'HCPCS_CD_15',
       'HCPCS_CD_16', 'HCPCS_CD_17', 'HCPCS_CD_18', 'HCPCS_CD_19',
       'HCPCS_CD_20', 'HCPCS_CD_21', 'HCPCS_CD_22', 'HC

In [71]:
for col in ['CLM_FROM_DT', 'CLM_THRU_DT']:
    outpatient[col] = pd.to_datetime(outpatient[col], format='%Y%m%d', errors='coerce')

outpatient["claim_duration (days)"] = (outpatient["CLM_THRU_DT"] - outpatient["CLM_FROM_DT"]).dt.total_seconds() / (3600 * 24)

median_duration = outpatient['claim_duration (days)'].median()
outpatient['claim_duration (days)'] = outpatient['claim_duration (days)'].fillna(median_duration)
outpatient["claim_duration (days)"].value_counts()

claim_duration (days)
0.0     696675
20.0     35288
1.0      17632
2.0       6212
14.0      3946
7.0       3870
3.0       3858
4.0       3230
5.0       2184
6.0       1997
9.0       1935
16.0      1727
8.0       1686
15.0      1473
11.0      1460
13.0      1458
10.0      1451
12.0      1295
17.0      1198
18.0      1128
19.0      1115
Name: count, dtype: int64

In [72]:
outpatient[['state_code', 'provider_block', 'unit_letter']] = outpatient['PRVDR_NUM'].apply(parse_prvdr_num)
outpatient['provider_category'] = outpatient.apply(categorize_provider, axis=1)

In [73]:
outpatient['year'] = outpatient['CLM_FROM_DT'].dt.year
# The entries with 2007 usually extend treatments into 2008 so lumping them together
outpatient['year'] = outpatient['year'].replace(2007, 2008)
outpatient["year"].value_counts(dropna=False)

year
2009.0    321900
2008.0    285480
2010.0    172216
NaN        11222
Name: count, dtype: int64

In [74]:
cols_to_check = [
    'DESYNPUF_ID', 'CLM_ID', 'SEGMENT', 'CLM_PMT_AMT', 'NCH_PRMRY_PYR_CLM_PD_AMT',
       'NCH_BENE_BLOOD_DDCTBL_LBLTY_AM', 'NCH_BENE_PTB_DDCTBL_AMT',
       'NCH_BENE_PTB_COINSRNC_AMT', 'ADMTNG_ICD9_DGNS_CD', 'ICD9_DGNS_CD_1', 'HCPCS_CD_1'
]

summaries = {}
for col in cols_to_check:
    summaries[col] = summarize_column(outpatient, col)

for col, summary in summaries.items():
    print(f"--- {col} ---")
    for k, v in summary.items():
        print(f"{k}: {v}")
    print("\n")


--- DESYNPUF_ID ---
missing_count: 0
missing_pct: 0.0
value_counts: DESYNPUF_ID
7AFDAE8C34E315DE    72
41F9479A2043F2E4    70
0B4482FF9A009EFA    69
4128A81BB8C13758    68
58CFC8B7149A9D1A    68
469D1D12B6699718    66
01E6F507A21824F9    66
1CC5CBC7C669E501    64
1F3231044F859948    64
804C5B131E04DF9B    63
Name: count, dtype: int64


--- CLM_ID ---
missing_count: 0
missing_pct: 0.0
min: 338012265475614
max: 338992266284594


--- SEGMENT ---
missing_count: 0
missing_pct: 0.0
min: 1
max: 2
value_counts: SEGMENT
1    779596
2     11222
Name: count, dtype: int64


--- CLM_PMT_AMT ---
missing_count: 0
missing_pct: 0.0
min: -100.0
max: 3300.0


--- NCH_PRMRY_PYR_CLM_PD_AMT ---
missing_count: 0
missing_pct: 0.0
min: 0.0
max: 14000.0


--- NCH_BENE_BLOOD_DDCTBL_LBLTY_AM ---
missing_count: 0
missing_pct: 0.0
min: 0.0
max: 700.0
value_counts: NCH_BENE_BLOOD_DDCTBL_LBLTY_AM
0.0      790789
200.0         9
100.0         9
400.0         5
300.0         3
700.0         2
600.0         1
Name: coun

In [75]:
# Check percentage of negative values
total = outpatient.shape[0]
neg_count = (outpatient['CLM_PMT_AMT'] < 0).sum()
neg_pct = neg_count / total * 100
print(f"CLM_PMT_AMT: {neg_count} negative values ({neg_pct:.4f}%)")

# Replace negatives with 0
outpatient['CLM_PMT_AMT'] = outpatient['CLM_PMT_AMT'].clip(lower=0)

CLM_PMT_AMT: 2615 negative values (0.3307%)


In [76]:
top_n = 10
top_codes_op = outpatient["ADMTNG_ICD9_DGNS_CD"].value_counts().head(top_n).index

def map_icd9_to_bucket_op(code):
    if pd.isna(code):
        return "missing"
    
    if code == "V7612":
        return "screening_prostate"
    
    if code in ["42731", "7862"]:
        return "cardiac"
    
    if code == "4019":
        return "hypertension"
    
    if code == "25000":
        return "diabetes"
    
    if code in ["V5883", "V5861"]:
        return "long_term_med_use"
    
    if code == "78900":
        return "GI"
    
    if code == "7295":
        return "musculoskeletal_pain"
    
    return "other"

outpatient["ADMTNG_BUCKET"] = outpatient["ADMTNG_ICD9_DGNS_CD"].apply(map_icd9_to_bucket_op)

In [77]:
outpatient["ADMTNG_BUCKET"].value_counts()

ADMTNG_BUCKET
missing                 595898
other                   156657
screening_prostate        8426
cardiac                   8197
long_term_med_use         6352
hypertension              4864
diabetes                  4444
GI                        3060
musculoskeletal_pain      2920
Name: count, dtype: int64

In [78]:
def map_icd9_to_bucket_op(code):
    if pd.isna(code) or code == "OTHER":
        return "missing"
    
    if code in ["4019", "4011"]:
        return "hypertension"
    
    if code in ["2720", "2721", "2722", "2723", "2724"]:
        return "lipid_disorder"
    
    if code in ["78650", "78651"]:
        return "respiratory"
    
    return "other"

outpatient["DGNS1_BUCKET"] = outpatient["ICD9_DGNS_CD_1"].apply(map_icd9_to_bucket_op)


In [79]:
# For outpatients, HCPCS is more meaningful than ICD
def map_hcpcs_bucket(code):
    if pd.isna(code):
        return "missing"
    
    if code == "36415":
        return "lab_draw"
    
    if code in ["99212", "99213", "99214"]:
        return "office_visit"
    
    if code in ["80053", "80048"]:
        return "lab_chemistry"
    
    if code == "A4657":
        return "dialysis_supply"
    
    if code == "85610":
        return "lab_coagulation"
    
    if code == "97110":
        return "therapy_rehab"
    
    if code == "G0202":
        return "imaging_screening"
    
    return "other"

outpatient["HCPCS_BUCKET"] = outpatient["HCPCS_CD_1"].apply(map_hcpcs_bucket)

In [80]:
# Dropping columns un-essential for answering the question
outpatient.drop(columns=["AT_PHYSN_NPI", "OP_PHYSN_NPI", "OT_PHYSN_NPI", 'ICD9_DGNS_CD_1', 'ICD9_DGNS_CD_2', 'ICD9_DGNS_CD_3', 'ICD9_DGNS_CD_4',
       'ICD9_DGNS_CD_5', 'ICD9_DGNS_CD_6', 'ICD9_DGNS_CD_7', 'ICD9_DGNS_CD_8',
       'ICD9_DGNS_CD_9', 'ICD9_DGNS_CD_10', 'ICD9_PRCDR_CD_1',
       'ICD9_PRCDR_CD_2', 'ICD9_PRCDR_CD_3', 'ICD9_PRCDR_CD_4',
       'ICD9_PRCDR_CD_5', 'ICD9_PRCDR_CD_6', 'HCPCS_CD_1',
       'HCPCS_CD_2', 'HCPCS_CD_3', 'HCPCS_CD_4', 'HCPCS_CD_5', 'HCPCS_CD_6',
       'HCPCS_CD_7', 'HCPCS_CD_8', 'HCPCS_CD_9', 'HCPCS_CD_10', 'HCPCS_CD_11',
       'HCPCS_CD_12', 'HCPCS_CD_13', 'HCPCS_CD_14', 'HCPCS_CD_15',
       'HCPCS_CD_16', 'HCPCS_CD_17', 'HCPCS_CD_18', 'HCPCS_CD_19',
       'HCPCS_CD_20', 'HCPCS_CD_21', 'HCPCS_CD_22', 'HCPCS_CD_23',
       'HCPCS_CD_24', 'HCPCS_CD_25', 'HCPCS_CD_26', 'HCPCS_CD_27',
       'HCPCS_CD_28', 'HCPCS_CD_29', 'HCPCS_CD_30', 'HCPCS_CD_31',
       'HCPCS_CD_32', 'HCPCS_CD_33', 'HCPCS_CD_34', 'HCPCS_CD_35',
       'HCPCS_CD_36', 'HCPCS_CD_37', 'HCPCS_CD_38', 'HCPCS_CD_39',
       'HCPCS_CD_40', 'HCPCS_CD_41', 'HCPCS_CD_42', 'HCPCS_CD_43',
       'HCPCS_CD_44', 'HCPCS_CD_45'], inplace=True)
outpatient.head()

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,NCH_BENE_BLOOD_DDCTBL_LBLTY_AM,NCH_BENE_PTB_DDCTBL_AMT,...,ADMTNG_ICD9_DGNS_CD,year,claim_duration (days),state_code,provider_block,unit_letter,provider_category,ADMTNG_BUCKET,DGNS1_BUCKET,HCPCS_BUCKET
0,000102649ED5601B,338982266168455,1,2008-06-15,2008-06-15,10016R,10.0,0.0,0.0,0.0,...,V5861,2008.0,0.0,10,100,None,Short-term hospital / ESRD clinic,long_term_med_use,other,lab_coagulation
1,000102649ED5601B,338322265737261,1,2008-06-19,2008-06-19,4901RP,50.0,0.0,0.0,0.0,...,78605,2008.0,0.0,49,490,None,Short-term hospital / ESRD clinic,other,other,other
2,000102649ED5601B,338522265531487,1,2008-08-18,2008-08-18,4901RP,300.0,0.0,0.0,0.0,...,NaN,2008.0,0.0,49,490,None,Short-term hospital / ESRD clinic,missing,other,missing
3,000102649ED5601B,338102266226170,1,2009-04-04,2009-04-04,0900SU,90.0,0.0,0.0,0.0,...,NaN,2009.0,0.0,09,090,None,Short-term hospital / ESRD clinic,missing,other,missing
4,000102649ED5601B,338482265489246,1,2009-07-01,2009-07-01,4900NA,50.0,0.0,0.0,0.0,...,73390,2009.0,0.0,49,490,None,Short-term hospital / ESRD clinic,other,other,other


In [81]:
outpatient.columns

Index(['DESYNPUF_ID', 'CLM_ID', 'SEGMENT', 'CLM_FROM_DT', 'CLM_THRU_DT',
       'PRVDR_NUM', 'CLM_PMT_AMT', 'NCH_PRMRY_PYR_CLM_PD_AMT',
       'NCH_BENE_BLOOD_DDCTBL_LBLTY_AM', 'NCH_BENE_PTB_DDCTBL_AMT',
       'NCH_BENE_PTB_COINSRNC_AMT', 'ADMTNG_ICD9_DGNS_CD', 'year',
       'claim_duration (days)', 'state_code', 'provider_block', 'unit_letter',
       'provider_category', 'ADMTNG_BUCKET', 'DGNS1_BUCKET', 'HCPCS_BUCKET'],
      dtype='object')

In [82]:
outpatient.drop(columns=['PRVDR_NUM', 'ADMTNG_ICD9_DGNS_CD', 'provider_block', 'unit_letter'], inplace=True)

In [83]:
outpatient.head()

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,NCH_BENE_BLOOD_DDCTBL_LBLTY_AM,NCH_BENE_PTB_DDCTBL_AMT,NCH_BENE_PTB_COINSRNC_AMT,year,claim_duration (days),state_code,provider_category,ADMTNG_BUCKET,DGNS1_BUCKET,HCPCS_BUCKET
0,000102649ED5601B,338982266168455,1,2008-06-15,2008-06-15,10.0,0.0,0.0,0.0,0.0,2008.0,0.0,10,Short-term hospital / ESRD clinic,long_term_med_use,other,lab_coagulation
1,000102649ED5601B,338322265737261,1,2008-06-19,2008-06-19,50.0,0.0,0.0,0.0,70.0,2008.0,0.0,49,Short-term hospital / ESRD clinic,other,other,other
2,000102649ED5601B,338522265531487,1,2008-08-18,2008-08-18,300.0,0.0,0.0,0.0,20.0,2008.0,0.0,49,Short-term hospital / ESRD clinic,missing,other,missing
3,000102649ED5601B,338102266226170,1,2009-04-04,2009-04-04,90.0,0.0,0.0,0.0,20.0,2009.0,0.0,09,Short-term hospital / ESRD clinic,missing,other,missing
4,000102649ED5601B,338482265489246,1,2009-07-01,2009-07-01,50.0,0.0,0.0,0.0,30.0,2009.0,0.0,49,Short-term hospital / ESRD clinic,other,other,other


In [84]:
for col in ['provider_category', 'ADMTNG_BUCKET', 'DGNS1_BUCKET', 'HCPCS_BUCKET']:
    outpatient[col] = outpatient[col].astype('category')

In [85]:
outpatient.dtypes

DESYNPUF_ID                               object
CLM_ID                                     int64
SEGMENT                                    int64
CLM_FROM_DT                       datetime64[ns]
CLM_THRU_DT                       datetime64[ns]
CLM_PMT_AMT                              float64
NCH_PRMRY_PYR_CLM_PD_AMT                 float64
NCH_BENE_BLOOD_DDCTBL_LBLTY_AM           float64
NCH_BENE_PTB_DDCTBL_AMT                  float64
NCH_BENE_PTB_COINSRNC_AMT                float64
year                                     float64
claim_duration (days)                    float64
state_code                                object
provider_category                       category
ADMTNG_BUCKET                           category
DGNS1_BUCKET                            category
HCPCS_BUCKET                            category
dtype: object

In [86]:
# Save outpatient
with open(os.path.join(folder, 'outpatient_cleaned.pkl'), 'wb') as f:
    pickle.dump(outpatient, f)

# **1.4 Carrier Claims**

In [102]:
carrier_a = pd.read_parquet("data/DE1_0_2008_to_2010_Carrier_Claims_Sample_9A_raw.parquet")
carrier_b = pd.read_parquet("data/DE1_0_2008_to_2010_Carrier_Claims_Sample_9B_raw.parquet")

In [103]:
carrier_a.head()

,DESYNPUF_ID,CLM_ID,CLM_FROM_DT,CLM_THRU_DT,ICD9_DGNS_CD_1,ICD9_DGNS_CD_2,ICD9_DGNS_CD_3,ICD9_DGNS_CD_4,ICD9_DGNS_CD_5,ICD9_DGNS_CD_6,...,LINE_ICD9_DGNS_CD_5,LINE_ICD9_DGNS_CD_6,LINE_ICD9_DGNS_CD_7,LINE_ICD9_DGNS_CD_8,LINE_ICD9_DGNS_CD_9,LINE_ICD9_DGNS_CD_10,LINE_ICD9_DGNS_CD_11,LINE_ICD9_DGNS_CD_12,LINE_ICD9_DGNS_CD_13,year
0,000102649ED5601B,684813370207372,20080301,20080301,2720,49121,V5869,None,None,None,...,2724,2724,2724,2724,2724,2724,2724,2724,2724,1970
1,000102649ED5601B,684253371655461,20080407,20080407,1741,V4571,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1970
2,000102649ED5601B,684323369524620,20080420,20080420,V285,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1970
3,000102649ED5601B,684163371235030,20080524,20080524,7850,4011,78079,None,None,None,...,None,None,None,None,None,None,None,None,None,1970
4,000102649ED5601B,684913369780400,20080604,20080604,33111,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1970


In [104]:
for col in ['CLM_FROM_DT', 'CLM_THRU_DT']:
    carrier_a[col] = pd.to_datetime(carrier_a[col], format='%Y%m%d', errors='coerce')
    carrier_b[col] = pd.to_datetime(carrier_b[col], format='%Y%m%d', errors='coerce')

carrier_a["claim_duration (days)"] = (carrier_a["CLM_THRU_DT"] - carrier_a["CLM_FROM_DT"]).dt.total_seconds() / (3600 * 24)
carrier_b["claim_duration (days)"] = (carrier_b["CLM_THRU_DT"] - carrier_b["CLM_FROM_DT"]).dt.total_seconds() / (3600 * 24)

median_duration_a = carrier_a['claim_duration (days)'].median()
median_duration_b = carrier_b['claim_duration (days)'].median()
carrier_a['claim_duration (days)'] = carrier_a['claim_duration (days)'].fillna(median_duration_a)
carrier_b['claim_duration (days)'] = carrier_b['claim_duration (days)'].fillna(median_duration_b)
carrier_a["claim_duration (days)"].value_counts()
carrier_b["claim_duration (days)"].value_counts()

claim_duration (days)
0.0    2242807
1.0      35759
2.0      21474
9.0      21321
3.0      14125
4.0      10246
5.0       7708
7.0       6662
6.0       4949
8.0       2216
Name: count, dtype: int64

In [105]:
carrier_a['year'] = carrier_a['CLM_FROM_DT'].dt.year
carrier_b['year'] = carrier_b['CLM_FROM_DT'].dt.year

# The entries with 2007 usually extend treatments into 2008 so lumping them together
carrier_a['year'] = carrier_a['year'].replace(2007, 2008)
carrier_b['year'] = carrier_b['year'].replace(2007, 2008)

carrier_a["year"].value_counts(dropna=False)
carrier_b["year"].value_counts(dropna=False)

year
2009    929485
2008    854965
2010    582817
Name: count, dtype: int64

In [106]:
def drop_unused_carrier_columns(df):
    """
    Drop unnecessary Carrier claim line columns:
    - PRF_PHYSN_NPI_1–13
    - TAX_NUM_1–13
    - HCPCS_CD_2–13
    - LINE_PRCSG_IND_CD_1–13
    - LINE_ICD9_DGNS_CD_1–13
    - ICD9_DGNS_CD_2-8
    """
    # Patterns of columns to drop
    patterns_to_drop = [
        "PRF_PHYSN_NPI_",
        "TAX_NUM_",
        "HCPCS_CD_",
        "LINE_PRCSG_IND_CD_",
        "LINE_ICD9_DGNS_CD_",
        "ICD9_DGNS_CD_"
    ]

    cols_to_drop = []
    for pattern in patterns_to_drop:
        # For HCPCS_CD_, we want only 2–13
        if pattern == "HCPCS_CD_":
            cols_to_drop += [col for col in df.columns if col.startswith(pattern) and int(col.split("_")[-1]) >= 2]
        # For ICD9_DGNS_CD_, we want only 2–13
        elif pattern == "ICD9_DGNS_CD_":
            cols_to_drop += [col for col in df.columns if col.startswith(pattern) and int(col.split("_")[-1]) >= 2]  
        else:
            cols_to_drop += [col for col in df.columns if col.startswith(pattern)]
    
    # Drop columns if they exist
    df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])
    return df

carrier_a_cleaned = drop_unused_carrier_columns(carrier_a)
carrier_b_cleaned = drop_unused_carrier_columns(carrier_b)

In [107]:
def summarize_carrier_payments(df):
    """
    Summarize line-level Carrier payment columns into totals.

    Computes:
    - total_line_nch_payment
    - total_line_allowed_charge
    - total_line_coinsurance
    - total_line_deductible
    - total_primary_payer_amount
    """

    # Identify line columns dynamically
    nch_cols = [col for col in df.columns if col.startswith("LINE_NCH_PMT_AMT_")]
    allowed_cols = [col for col in df.columns if col.startswith("LINE_ALOWD_CHRG_AMT_")]
    coins_cols = [col for col in df.columns if col.startswith("LINE_COINSRNC_AMT_")]
    deduct_cols = [col for col in df.columns if col.startswith("LINE_BENE_PTB_DDCTBL_AMT_")]
    primary_cols = [col for col in df.columns if col.startswith("LINE_BENE_PRMRY_PYR_PD_AMT_")]


    # Sum across lines (skip NaNs)
    df["total_line_nch_payment"] = df[nch_cols].sum(axis=1, skipna=True)
    df["total_line_allowed_charge"] = df[allowed_cols].sum(axis=1, skipna=True)
    df["total_line_coinsurance"] = df[coins_cols].sum(axis=1, skipna=True)
    df["total_line_deductible"] = df[deduct_cols].sum(axis=1, skipna=True)
    df["total_primary_payer_amount"] = df[primary_cols].sum(axis=1, skipna=True)


    return df


In [108]:
carrier_a = summarize_carrier_payments(carrier_a_cleaned)
carrier_b = summarize_carrier_payments(carrier_b_cleaned)

In [109]:
def drop_line_payment_columns(df):
    """
    Drop all individual line-level payment columns after computing totals.
    """
    patterns_to_drop = [
        "LINE_NCH_PMT_AMT_",
        "LINE_BENE_PTB_DDCTBL_AMT_",
        "LINE_BENE_PRMRY_PYR_PD_AMT_",
        "LINE_COINSRNC_AMT_",
        "LINE_ALOWD_CHRG_AMT_"
    ]
    
    cols_to_drop = []
    for pattern in patterns_to_drop:
        cols_to_drop += [col for col in df.columns if col.startswith(pattern)]
    
    # Drop columns if they exist
    df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])
    return df

In [110]:
carrier_a_cleaned = drop_line_payment_columns(carrier_a)
carrier_b_cleaned = drop_line_payment_columns(carrier_b)

In [111]:
carrier_b_cleaned.columns

Index(['DESYNPUF_ID', 'CLM_ID', 'CLM_FROM_DT', 'CLM_THRU_DT', 'ICD9_DGNS_CD_1',
       'HCPCS_CD_1', 'year', 'claim_duration (days)', 'total_line_nch_payment',
       'total_line_allowed_charge', 'total_line_coinsurance',
       'total_line_deductible', 'total_primary_payer_amount'],
      dtype='object')

In [112]:
cols_to_check = ['ICD9_DGNS_CD_1', 'HCPCS_CD_1']
summaries = {}

for col in cols_to_check:
    summaries[col] = {
        'carrier_a': summarize_column(carrier_a_cleaned, col),
        'carrier_b': summarize_column(carrier_b_cleaned, col)
    }

for col, datasets in summaries.items():
    print(f"--- {col} ---")
    for dataset_name, summary in datasets.items():
        print(f"{dataset_name}:")
        for k, v in summary.items():
            print(f"  {k}: {v}")
    print("\n")

--- ICD9_DGNS_CD_1 ---
carrier_a:
  missing_count: 69
  missing_pct: 0.002914754826876236
  value_counts: ICD9_DGNS_CD_1
4019     56192
4011     56121
78651    14516
78650    14449
78659    14408
7802     11377
2723     10785
2722     10710
2724     10688
2721     10654
Name: count, dtype: int64
carrier_b:
  missing_count: 64
  missing_pct: 0.002703539566935204
  value_counts: ICD9_DGNS_CD_1
4019     56057
4011     55828
78650    14620
78651    14512
78659    14457
7802     11507
2723     10762
2722     10653
2720     10594
2721     10585
Name: count, dtype: int64


--- HCPCS_CD_1 ---
carrier_a:
  missing_count: 26804
  missing_pct: 1.1322766431824731
  value_counts: HCPCS_CD_1
99213    285659
99214    205556
36415     75189
99232     56449
99212     54409
71010     48255
93010     41406
71020     36186
98941     31165
92014     30462
Name: count, dtype: int64
carrier_b:
  missing_count: 27005
  missing_pct: 1.1407669688294562
  value_counts: HCPCS_CD_1
99213    285274
99214    206290


In [113]:
# Carrier A mapping
def map_icd9_to_bucket_op_a(code):
    if pd.isna(code) or code == "OTHER":
        return "missing"
    
    if code in ["4019", "4011"]:
        return "hypertension"
    
    if code in ["2720", "2721", "2722", "2723", "2724"]:
        return "lipid_disorder"
    
    if code in ["78650", "78651", "78659"]:
        return "respiratory"
    
    if code in ["7802"]:
        return "neurologic"
    
    return "other"

# Carrier B mapping
def map_icd9_to_bucket_op_b(code):
    if pd.isna(code) or code == "OTHER":
        return "missing"
    
    if code in ["4019", "4011"]:
        return "hypertension"
    
    if code in ["2720", "2721", "2722", "2723", "2724"]:
        return "lipid_disorder"
    
    if code in ["78650", "78651", "78659"]:
        return "respiratory"
    
    if code in ["7802"]:
        return "neurologic"
    
    if code in ["2720"]:  # if you want, include any extra carrier-B-specific code
        return "other_special"
    
    return "other"

# Apply
carrier_a_cleaned["DGNS1_BUCKET"] = carrier_a_cleaned["ICD9_DGNS_CD_1"].apply(map_icd9_to_bucket_op_a)
carrier_b_cleaned["DGNS1_BUCKET"] = carrier_b_cleaned["ICD9_DGNS_CD_1"].apply(map_icd9_to_bucket_op_b)


In [114]:
# Carrier A top HCPCS codes
top_hcpcs_a = [
    "99213", "99214", "36415", "99232", "99212", 
    "71010", "93010", "71020", "98941", "92014"
]

def map_hcpcs_to_bucket_a(code):
    if pd.isna(code):
        return "missing"
    
    if code in ["99212", "99213", "99214", "99232"]:
        return "office_visit"
    
    if code == "36415":
        return "lab_blood_draw"
    
    if code in ["71010", "71020", "92014"]:
        return "imaging"
    
    if code == "93010":
        return "cardiac_procedure"
    
    if code == "98941":
        return "therapy"
    
    return "other"

# Carrier B top HCPCS codes
top_hcpcs_b = [
    "99213", "99214", "36415", "99232", "99212", 
    "71010", "93010", "71020", "98941", "92014"
]

def map_hcpcs_to_bucket_b(code):
    if pd.isna(code):
        return "missing"
    
    if code in ["99212", "99213", "99214", "99232"]:
        return "office_visit"
    
    if code == "36415":
        return "lab_blood_draw"
    
    if code in ["71010", "71020", "92014"]:
        return "imaging"
    
    if code == "93010":
        return "cardiac_procedure"
    
    if code == "98941":
        return "therapy"
    
    return "other"

# Apply
carrier_a_cleaned["HCPCS_BUCKET"] = carrier_a_cleaned["HCPCS_CD_1"].apply(map_hcpcs_to_bucket_a)
carrier_b_cleaned["HCPCS_BUCKET"] = carrier_b_cleaned["HCPCS_CD_1"].apply(map_hcpcs_to_bucket_b)


In [115]:
carrier_a_cleaned.head()

,DESYNPUF_ID,CLM_ID,CLM_FROM_DT,CLM_THRU_DT,ICD9_DGNS_CD_1,HCPCS_CD_1,year,claim_duration (days),total_line_nch_payment,total_line_allowed_charge,total_line_coinsurance,total_line_deductible,total_primary_payer_amount,DGNS1_BUCKET,HCPCS_BUCKET
0,000102649ED5601B,684813370207372,2008-03-01,2008-03-01,2720,71020,2008,0.0,40.0,40.0,0.0,0.0,0.0,lipid_disorder,imaging
1,000102649ED5601B,684253371655461,2008-04-07,2008-04-07,1741,99213,2008,0.0,40.0,110.0,10.0,0.0,0.0,other,office_visit
2,000102649ED5601B,684323369524620,2008-04-20,2008-04-20,V285,77057,2008,0.0,60.0,30.0,10.0,0.0,0.0,other,other
3,000102649ED5601B,684163371235030,2008-05-24,2008-05-24,7850,93010,2008,0.0,60.0,240.0,50.0,0.0,0.0,other,cardiac_procedure
4,000102649ED5601B,684913369780400,2008-06-04,2008-06-04,33111,99308,2008,0.0,40.0,80.0,10.0,0.0,0.0,other,other


In [116]:
carrier_a_cleaned.drop(columns=["ICD9_DGNS_CD_1", "HCPCS_CD_1"], inplace = True)

In [117]:
carrier_b_cleaned.drop(columns=["ICD9_DGNS_CD_1", "HCPCS_CD_1"], inplace = True)

In [118]:
for col in ['DGNS1_BUCKET', 'HCPCS_BUCKET']:
    carrier_a_cleaned[col] = carrier_a_cleaned[col].astype('category')
    carrier_b_cleaned[col] = carrier_b_cleaned[col].astype('category')

In [119]:
carrier_b_cleaned.dtypes

DESYNPUF_ID                           object
CLM_ID                                 int64
CLM_FROM_DT                   datetime64[ns]
CLM_THRU_DT                   datetime64[ns]
year                                   int32
claim_duration (days)                float64
total_line_nch_payment               float64
total_line_allowed_charge            float64
total_line_coinsurance               float64
total_line_deductible                float64
total_primary_payer_amount           float64
DGNS1_BUCKET                        category
HCPCS_BUCKET                        category
dtype: object

In [120]:
# Save carrier
with open(os.path.join(folder, 'carrier_a_cleaned.pkl'), 'wb') as f:
    pickle.dump(carrier_a_cleaned, f)
with open(os.path.join(folder, 'carrier_b_cleaned.pkl'), 'wb') as f:
    pickle.dump(carrier_b_cleaned, f)

# **1.5 Prescription Drug Events**

In [121]:
pde = pd.read_pickle("data/pde_raw.pkl")

In [122]:
pde.head()

,DESYNPUF_ID,PDE_ID,SRVC_DT,PROD_SRVC_ID,QTY_DSPNSD_NUM,DAYS_SUPLY_NUM,PTNT_PAY_AMT,TOT_RX_CST_AMT,year
0,000102649ED5601B,30064474910782,20080213,53489014401,30.0,30,0.0,0.0,1970
1,000102649ED5601B,30184475399677,20080216,65243012103,30.0,90,0.0,10.0,1970
2,000102649ED5601B,30954474861797,20080303,55289072530,30.0,30,0.0,0.0,1970
3,000102649ED5601B,30164475487014,20080307,67228029606,30.0,90,10.0,20.0,1970
4,000102649ED5601B,30344475127285,20080417,51655074524,60.0,30,0.0,30.0,1970


In [123]:
for col in ['SRVC_DT']:
    pde[col] = pd.to_datetime(pde[col], format='%Y%m%d', errors='coerce')

In [124]:
pde['year'] = pde['SRVC_DT'].dt.year
pde["year"].value_counts(dropna=False)

year
2009    2170505
2008    1994008
2010    1387957
Name: count, dtype: int64

In [125]:
cols_to_check = [
    "PDE_ID", "PROD_SRVC_ID", "QTY_DSPNSD_NUM", "DAYS_SUPLY_NUM", "PTNT_PAY_AMT", "TOT_RX_CST_AMT"
]

summaries = {}
for col in cols_to_check:
    summaries[col] = summarize_column(pde, col)

for col, summary in summaries.items():
    print(f"--- {col} ---")
    for k, v in summary.items():
        print(f"{k}: {v}")
    print("\n")

--- PDE_ID ---
missing_count: 0
missing_pct: 0.0
min: 30014473231884
max: 30994478915988


--- PROD_SRVC_ID ---
missing_count: 0
missing_pct: 0.0
value_counts: PROD_SRVC_ID
00037024110    192
00037024105    185
00247219630    185
00002897101    184
00002840099    184
54868604900    182
54868395700    181
52959081303    180
62381897101    178
54868540600    176
Name: count, dtype: int64


--- QTY_DSPNSD_NUM ---
missing_count: 0
missing_pct: 0.0
min: 0.0
max: 360.0


--- DAYS_SUPLY_NUM ---
missing_count: 0
missing_pct: 0.0
min: 0
max: 90
value_counts: DAYS_SUPLY_NUM
30    3975111
90     597190
10     512223
20     232422
0      117559
60      55391
50      26447
80      21901
40      11735
70       2491
Name: count, dtype: int64


--- PTNT_PAY_AMT ---
missing_count: 0
missing_pct: 0.0
min: 0.0
max: 170.0


--- TOT_RX_CST_AMT ---
missing_count: 0
missing_pct: 0.0
min: 0.0
max: 570.0




In [126]:
top_ndcs = [
    "00037024110", "00037024105", "00247219630", "00002897101",
    "00002840099", "54868604900", "54868395700", "52959081303",
    "62381897101", "54868540600"
]

def map_ndc_to_bucket(ndc):
    if pd.isna(ndc):
        return "missing"
    
    if ndc in ["00037024110", "00037024105"]:
        return "allergy"          
    if ndc == "00247219630":
        return "antihypertensive"
    if ndc in ["00002897101", "00002840099"]:
        return "lipid_disorder"
    if ndc in ["54868604900", "54868395700"]:
        return "antidiabetic"
    if ndc in ["52959081303", "62381897101", "54868540600"]:
        return "pain_management"
    
    return "other"

pde["NDC_BUCKET"] = pde["PROD_SRVC_ID"].apply(map_ndc_to_bucket).astype("category")


In [127]:
pde.drop(columns=["PROD_SRVC_ID"], inplace=True)

In [128]:
pde.dtypes

DESYNPUF_ID               object
PDE_ID                     int64
SRVC_DT           datetime64[ns]
QTY_DSPNSD_NUM           float64
DAYS_SUPLY_NUM             int64
PTNT_PAY_AMT             float64
TOT_RX_CST_AMT           float64
year                       int32
NDC_BUCKET              category
dtype: object

In [129]:
# Save pde
with open(os.path.join(folder, 'pde_cleaned.pkl'), 'wb') as f:
    pickle.dump(pde, f)